# 2D Hydrodynamics Simulation

This notebook is the interface that you can use to set up the parameters, solve the equations, and visualize the results for the 2D hydrodynamics simulation. 

## Import modules and set up parameters:
Import relevant modules for the simulations:

In [33]:
import rusanov_flux as flux
import utils
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, writers
from IPython import display 

Users define relevant parameters for simulation: 

In [34]:
# Simulation parameters (change as needed)
N                      = 128 # resolution
boxsize                = 1. # size of the box
gamma                  = 5/3 # ideal gas gamma
courant_fac            = 0.4 # Courant number
tEnd                   = 15 # end time (start time is 0)
tOut                   = 0.02 # draw frequency
plotRealTime           = True # switch on for visualizing the simulation as a video
gravity                = False 
ghostCell              = False

Users define initial conditions for simulation:
Two cells have been set up for two different tests:
- test 1: Kelvin-Helmholtz Instability
- test 2: Rayleigh-Taylor Instability

In [3]:
#  Run this cell for test 1: Kelvin-Helmholtz Instability
gravity = False 
ghostCell  = False

def initialConditions(X, Y, g = None):
    w0 = 0.1
    sigma = 0.05/np.sqrt(2.)
    rho = 1. + (np.abs(Y-0.5) < 0.25)
    vx = -0.5 + (np.abs(Y-0.5)<0.25)
    vy = w0*np.sin(4*np.pi*X) * ( np.exp(-(Y-0.25)**2/(2 * sigma**2)) + np.exp(-(Y-0.75)**2/(2*sigma**2)) )
    P = 2.5 * np.ones(X.shape)
    return rho, vx, vy, P, g

In [35]:
# Run this cell for test 2: Rayleigh-Taylor Instability
gravity = True
ghostCell = True

def initialConditions(X, Y, g = -0.1):
    w0 = 0.0025
    P0 = 2.5
    rho = 1. + (Y > 0.75)
    vx = np.zeros(X.shape)
    vy = w0 * (1-np.cos(4*np.pi*X)) * (1-np.cos(4*np.pi*Y/3)) 
    P = P0 + g * (Y-0.75) * rho
    return rho, vx, vy, P, g

### Calculation starts here (DO NOT change parameters below):
Set up grid and parameters needed for calcualtions

In [36]:
# Mesh
t = 0
dx = boxsize / N
vol = dx**2
xlin = np.linspace(0.5*dx, boxsize-0.5*dx, N)
Y, X = np.meshgrid( xlin, xlin )
rho, vx, vy, P, g= initialConditions(X, Y)
if ghostCell:
    rho, vx, vy, P = utils.addGhostCells(rho, vx, vy, P)

# Get conserved variables
Mass, Momx, Momy, Energy = utils.getConserved( rho, vx, vy, P, gamma, vol )


In [37]:
outputCount = 1
frames = []
while t < tEnd:
    
    # get Primitive variables
        rho, vx, vy, P = utils.getPrimitive( Mass, Momx, Momy, Energy, gamma, vol, ghostCell)
        
        # get time step (CFL) = dx / max signal speed
        dt = courant_fac * np.min( dx / (np.sqrt( gamma*P/rho ) + np.sqrt(vx**2+vy**2)) )
        
        if gravity:
            Mass, Momx, Momy, Energy = utils.addSourceTerm( Mass, Momx, Momy, Energy, g, dt/2 )
		# get Primitive variables
            rho, vx, vy, P = utils.getPrimitive( Mass, Momx, Momy, Energy, gamma, vol, ghostCell )

        # calculate gradients
        rho_dx, rho_dy = utils.getGradient(rho, dx, ghostCell)
        vx_dx,  vx_dy  = utils.getGradient(vx,  dx, ghostCell)
        vy_dx,  vy_dy  = utils.getGradient(vy,  dx, ghostCell)
        P_dx,   P_dy   = utils.getGradient(P,   dx, ghostCell)
        
        # extrapolate half-step in time
        rho_prime = rho - 0.5*dt * ( vx * rho_dx + rho * vx_dx + vy * rho_dy + rho * vy_dy)
        vx_prime  = vx  - 0.5*dt * ( vx * vx_dx + vy * vx_dy + (1/rho) * P_dx )
        vy_prime  = vy  - 0.5*dt * ( vx * vy_dx + vy * vy_dy + (1/rho) * P_dy )
        P_prime   = P   - 0.5*dt * ( gamma*P * (vx_dx + vy_dy)  + vx * P_dx + vy * P_dy )
        
        # extrapolate in space to face centers
        rho_XL, rho_XR, rho_YL, rho_YR = utils.extrapolateInSpaceToFace(rho_prime, rho_dx, rho_dy, dx)
        vx_XL,  vx_XR,  vx_YL,  vx_YR  = utils.extrapolateInSpaceToFace(vx_prime,  vx_dx,  vx_dy,  dx)
        vy_XL,  vy_XR,  vy_YL,  vy_YR  = utils.extrapolateInSpaceToFace(vy_prime,  vy_dx,  vy_dy,  dx)
        P_XL,   P_XR,   P_YL,   P_YR   = utils.extrapolateInSpaceToFace(P_prime,   P_dx,   P_dy,   dx)
        
        # compute fluxes (local Lax-Friedrichs/Rusanov)
        flux_Mass_X, flux_Momx_X, flux_Momy_X, flux_Energy_X = flux.getFlux(rho_XL, rho_XR, vx_XL, vx_XR, vy_XL, vy_XR, P_XL, P_XR, gamma)
        flux_Mass_Y, flux_Momy_Y, flux_Momx_Y, flux_Energy_Y = flux.getFlux(rho_YL, rho_YR, vy_YL, vy_YR, vx_YL, vx_YR, P_YL, P_YR, gamma)
        
        # update solution
        Mass   = flux.applyFluxes(Mass, flux_Mass_X, flux_Mass_Y, dx, dt)
        Momx   = flux.applyFluxes(Momx, flux_Momx_X, flux_Momx_Y, dx, dt)
        Momy   = flux.applyFluxes(Momy, flux_Momy_X, flux_Momy_Y, dx, dt)
        Energy = flux.applyFluxes(Energy, flux_Energy_X, flux_Energy_Y, dx, dt)

        if gravity:
            Mass, Momx, Momy, Energy = utils.addSourceTerm( Mass, Momx, Momy, Energy, g, dt/2 )
        
        # update time
        t += dt
        if plotRealTime or t >= tEnd:
            if(outputCount%10 == 0):
                frames.append(rho.T)
            outputCount += 1




In [38]:
# Create an animation from the stored frames
fig, ax = plt.subplots()

def animate(i):
    plt.cla()
    plt.imshow(frames[i],cmap="plasma")
    plt.clim(0.8, 2.2)
    ax = plt.gca()
    ax.invert_yaxis()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)	
    ax.set_aspect('equal')

ani = FuncAnimation(fig, animate, frames=len(frames), interval=10, repeat=False)
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()